In [1]:
# =========================
# 0) Colab Setup
# =========================
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

pd.set_option("display.max_columns", 200)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print("Setup complete.")


Setup complete.


In [ ]:
# =========================
# 1) Download dataset from Kaggle (requires kaggle.json)
# =========================
# Option 1 (recommended): Upload kaggle.json
# In Colab: left sidebar -> Files -> upload kaggle.json
#
# Then run this cell.

!pip -q install kaggle

# Make Kaggle directory
!mkdir -p ~/.kaggle

# Move kaggle.json if you uploaded it to /content
if os.path.exists("/content/kaggle.json"):
    !cp /content/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

# Download dataset
# Kaggle dataset: austinreese/craigslist-carstrucks-data
!kaggle datasets download -d austinreese/craigslist-carstrucks-data -p /content --unzip

print("Download complete. Listing files:")
!ls -lah /content


Dataset URL: https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data
License(s): CC0-1.0
 51% 135M/262M [00:00<00:00, 1.40GB/s]
100% 262M/262M [00:00<00:00, 652MB/s] 
Download complete. Listing files:
total 1.4G
drwxr-xr-x 1 root root 4.0K Dec 18 22:56 .
drwxr-xr-x 1 root root 4.0K Dec 18 22:26 ..
drwxr-xr-x 4 root root 4.0K Dec 11 14:34 .config
-rw-r--r-- 1 root root   89 Dec 18 22:56 kaggle.json
drwxr-xr-x 1 root root 4.0K Dec 11 14:34 sample_data
-rw-r--r-- 1 root root 1.4G Dec 18 22:56 vehicles.csv
